In [10]:
# Import the appropriate packages & modules
import snowflake.connector
from snowflake.connector.converter_null import SnowflakeNoConverterToPython
import pandas as pd 
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
import sqlalchemy as sa
import os

In [11]:
# Set some variables for the account, user & Password
# Modify this section to match your demo account
# and create an 'engine' for the Snowflake connection
ACCOUNT = os.getenv('SNOW_ACCOUNT')
USER = os.getenv('SNOW_USER')
PASSWORD = os.getenv('SNOW_PASSWORD')

engine = create_engine(URL(
    account = ACCOUNT,
    user = USER,
    password = PASSWORD,
    database = os.getenv('SNOW_DATABASE'),
    schema = os.getenv('SNOW_SCHEMA') or 'public',
    warehouse = os.getenv('SNOW_WAREHOUSE'),
    role= os.getenv('SNOW_ROLE'),
))


In [12]:
connection = engine.connect()
try:
    sql = "create or replace table sample_product_data (id int, parent_id int, category_id int, name varchar, serial_number varchar, key int, \"3rd\" int);"
    connection.execute(sql)
    sql = """insert into sample_product_data values
    (1, 0, 5, 'Product 1', 'prod-1', 1, 10),
    (2, 1, 5, 'Product 1A', 'prod-1-A', 1, 20),
    (3, 1, 5, 'Product 1B', 'prod-1-B', 1, 30),
    (4, 0, 10, 'Product 2', 'prod-2', 2, 40),
    (5, 4, 10, 'Product 2A', 'prod-2-A', 2, 50),
    (6, 4, 10, 'Product 2B', 'prod-2-B', 2, 60),
    (7, 0, 20, 'Product 3', 'prod-3', 3, 70),
    (8, 7, 20, 'Product 3A', 'prod-3-A', 3, 80),
    (9, 7, 20, 'Product 3B', 'prod-3-B', 3, 90),
    (10, 0, 50, 'Product 4', 'prod-4', 4, 100),
    (11, 10, 50, 'Product 4A', 'prod-4-A', 4, 100),
    (12, 10, 50, 'Product 4B', 'prod-4-B', 4, 100);"""
    connection.execute(sql)
finally:
    connection.close()
    engine.dispose()

In [13]:
sql = "select * from sample_product_data limit 1000"
# Use Pandas dataframe method read_sql_query to execute SQL in SQL Alchemy 
#%%time
df = pd.read_sql_query(sql, engine)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             12 non-null     int64 
 1   parent_id      12 non-null     int64 
 2   category_id    12 non-null     int64 
 3   name           12 non-null     object
 4   serial_number  12 non-null     object
 5   key            12 non-null     int64 
 6   3rd            12 non-null     int64 
dtypes: int64(5), object(2)
memory usage: 800.0+ bytes


In [15]:
pd.isnull(df).any()

id               False
parent_id        False
category_id      False
name             False
serial_number    False
key              False
3rd              False
dtype: bool

In [16]:
df.groupby('category_id').count()

,id,parent_id,name,serial_number,key,3rd
category_id,,,,,,
5,3,3,3,3,3,3
10,3,3,3,3,3,3
20,3,3,3,3,3,3
50,3,3,3,3,3,3
